# EEL891 - Trabalho 1
## Classificador para apoio à decisão de aprovação de crédito

In [256]:
import pandas as pd

In [257]:
dados_treino = pd.read_csv('conjunto_de_treinamento.csv')
dados_teste = pd.read_csv('conjunto_de_teste.csv')

In [258]:
# Visualizar as colunas
dados_treino.T

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
id_solicitante,1,2,3,4,5,6,7,8,9,10,...,19991,19992,19993,19994,19995,19996,19997,19998,19999,20000
produto_solicitado,1,1,1,1,7,1,1,1,2,1,...,1,1,1,1,2,1,1,1,1,2
dia_vencimento,10,25,20,20,1,20,15,5,25,10,...,10,10,5,5,5,10,20,10,5,20
forma_envio_solicitacao,presencial,internet,internet,internet,internet,presencial,presencial,internet,internet,presencial,...,presencial,presencial,internet,internet,presencial,presencial,presencial,internet,internet,presencial
tipo_endereco,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
sexo,M,F,F,M,F,M,F,F,F,M,...,F,M,M,F,M,M,F,F,F,F
idade,85,38,37,37,51,21,64,20,39,44,...,52,48,62,18,23,27,26,63,84,53
estado_civil,2,1,2,1,1,1,4,1,2,2,...,4,2,4,1,2,2,2,2,1,1
qtde_dependentes,0,0,0,1,3,1,2,0,2,2,...,0,2,0,0,0,0,1,0,0,0
grau_instrucao,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [259]:
# Juntar os dois DataFrames para manipular os dados
dados = pd.concat([dados_treino, dados_teste])
dados.shape

(25000, 42)

In [260]:
# Transformar o id_solicitante no index para que ele não seja considerado no modelo
dados = dados.set_index(dados['id_solicitante'])

In [261]:
# Observar os diferentes valores em cada coluna
for v in dados.columns:
    print('%10s:' % v, '%4d categorias' % len((dados_treino[v].unique())))
    print(dados_treino[v].unique(), '\n')

id_solicitante: 20000 categorias
[    1     2     3 ... 19998 19999 20000] 

produto_solicitado:    3 categorias
[1 7 2] 

dia_vencimento:    6 categorias
[10 25 20  1 15  5] 

forma_envio_solicitacao:    3 categorias
['presencial' 'internet' 'correio'] 

tipo_endereco:    2 categorias
[1 2] 

      sexo:    4 categorias
['M' 'F' 'N' ' '] 

     idade:   84 categorias
[ 85  38  37  51  21  64  20  39  44  23  29  28  33  40  41  54  30  19
  45  32  47  36  56  50  31  53  60  35  57  42  43  49  25  34  65  72
  83  48  61  52  66  75  59  22  26  18  68  24  73  58  69  27  79  46
  80  55  70  95  62  74  89  67  17 106  77  63  81  82  76  71  90  87
  78  86  84  91  88   7  96  94  97  92  93 100] 

estado_civil:    8 categorias
[2 1 4 6 7 5 3 0] 

qtde_dependentes:   15 categorias
[ 0  1  3  2  7  5  6  4 11  8  9 10 13 53 14] 

grau_instrucao:    1 categorias
[0] 

nacionalidade:    3 categorias
[1 0 2] 

estado_onde_nasceu:   28 categorias
['CE' 'SE' 'BA' 'RS' 'SP' 'ES' 'GO' '

### Observando as variáveis podemos observar que:

- `forma_envio_solicitacao`: não-ordinal com 3 categorias -> ONE HOT ENCONDING
- `sexo`: não-ordinal com 3 categorias -> CONVERTER 
- `grau_instrucao`: 1 categoria -> DESCARTAR
- `estado_onde_nasceu`: não-ordinal com 27 categorias -> DESCARTAR
- `estado_onde reside`: não-ordinal com 27 categorias -> DESCARTAR
- `possui_telefone_residencial`: binária -> CONVERTER
- `codigo_area_telefone_residencial`: não-ordinal com 81 categorias -> DESCARTAR
- `possui_telefone_celular`: 1 categoria -> DESCARTAR
- `vinculo_formal_com_empresa`: binária -> CONVERTER
- `estado_onde_trabalha`: não-ordinal com 27 categorias -> DESCARTAR
- `possui_telefone_trabalha`: binária -> CONVERTER
- `codigo_area_telefone_trabalho`: não-ordinal com 77 categorias -> DESCARTAR

In [262]:
# Descartando variáveis
dados = dados.drop(['id_solicitante', 'grau_instrucao', 'estado_onde_nasceu', 'estado_onde_reside',
                                  'estado_onde_trabalha', 'codigo_area_telefone_residencial',
                                  'possui_telefone_celular', 'codigo_area_telefone_trabalho'], axis=1)

In [263]:
dados['sexo'].value_counts()

F    15296
M     9664
N       29
        11
Name: sexo, dtype: int64

In [264]:
# Vemos que os valores diferentes de M e F são apenas 40, então vou retirar tais linhas
dados_treino = dados_treino[dados_treino.sexo != 'N']
dados_treino = dados_treino[dados_treino.sexo != ' ']

In [265]:
# Aplicar binarização simples nas variáveis que tenham apenas duas categorias
from sklearn.preprocessing import LabelBinarizer

binarizador = LabelBinarizer()

for v in ['sexo', 'possui_telefone_residencial', 'vinculo_formal_com_empresa', 'possui_telefone_trabalho']:
    dados[v] = binarizador.fit_transform(dados[v])
    
dados.T

id_solicitante,1,2,3,4,5,6,7,8,9,10,...,24991,24992,24993,24994,24995,24996,24997,24998,24999,25000
produto_solicitado,1,1,1,1,7,1,1,1,2,1,...,1,7,1,1,1,1,1,1,1,1
dia_vencimento,10,25,20,20,1,20,15,5,25,10,...,10,5,15,20,10,10,5,10,25,25
forma_envio_solicitacao,presencial,internet,internet,internet,internet,presencial,presencial,internet,internet,presencial,...,internet,internet,presencial,internet,presencial,presencial,internet,presencial,internet,presencial
tipo_endereco,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
sexo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
idade,85,38,37,37,51,21,64,20,39,44,...,30,38,21,34,85,49,18,29,31,54
estado_civil,2,1,2,1,1,1,4,1,2,2,...,2,1,2,1,4,2,2,1,2,1
qtde_dependentes,0,0,0,1,3,1,2,0,2,2,...,0,0,0,0,0,1,0,0,2,0
nacionalidade,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
possui_telefone_residencial,1,1,1,1,1,1,1,1,1,0,...,1,1,1,1,1,0,1,1,1,1


In [266]:
# Aplicar one-hot enconding na variável "area_code"
dados = pd.get_dummies(dados, columns=['forma_envio_solicitacao'])
dados.columns

Index(['produto_solicitado', 'dia_vencimento', 'tipo_endereco', 'sexo',
       'idade', 'estado_civil', 'qtde_dependentes', 'nacionalidade',
       'possui_telefone_residencial', 'tipo_residencia', 'meses_na_residencia',
       'possui_email', 'renda_mensal_regular', 'renda_extra',
       'possui_cartao_visa', 'possui_cartao_mastercard',
       'possui_cartao_diners', 'possui_cartao_amex', 'possui_outros_cartoes',
       'qtde_contas_bancarias', 'qtde_contas_bancarias_especiais',
       'valor_patrimonio_pessoal', 'possui_carro',
       'vinculo_formal_com_empresa', 'possui_telefone_trabalho',
       'meses_no_trabalho', 'profissao', 'ocupacao', 'profissao_companheiro',
       'grau_instrucao_companheiro', 'local_onde_reside',
       'local_onde_trabalha', 'inadimplente',
       'forma_envio_solicitacao_correio', 'forma_envio_solicitacao_internet',
       'forma_envio_solicitacao_presencial'],
      dtype='object')

In [267]:
# Substituir os valores NaN por 0
dados.fillna(0, inplace=True)

In [268]:
# Verificar o valor médio de cada atributo em cada classe e ver o quanto cada variável 
# influencia. Se a média for parecia do True e False, ela provavelmente não influencia.
dados.groupby(['inadimplente']).mean().T

inadimplente,0.0,1.0
produto_solicitado,1.268733,1.314700
dia_vencimento,12.789467,13.689400
tipo_endereco,1.006400,1.006000
sexo,0.000400,0.000500
idade,43.504000,40.550800
estado_civil,2.151733,2.079200
qtde_dependentes,0.642133,0.688200
nacionalidade,0.961200,0.961700
possui_telefone_residencial,0.848533,0.792900
tipo_residencia,1.220200,1.243800


In [269]:
# Selecionar os atributos que serão utilizados pelo classificador 
atributos_selecionados = ['produto_solicitado', 
                          'dia_vencimento',
                          'tipo_endereco', 
                          'sexo', 
                          'idade', 
                          'estado_civil', 
                          'qtde_dependentes',
                          'nacionalidade', 
                          'possui_telefone_residencial', 
                          'tipo_residencia',
                          'meses_na_residencia', 
                          'possui_email', 
                          'renda_mensal_regular',
                          'renda_extra', 
                          'possui_cartao_visa', 
                          'possui_cartao_mastercard',
                          'possui_cartao_diners', 
                          'possui_cartao_amex', 
                          'possui_outros_cartoes',
                          'qtde_contas_bancarias', 
                          'qtde_contas_bancarias_especiais',
                          'valor_patrimonio_pessoal', 
                          'possui_carro',
                          'vinculo_formal_com_empresa', 
                          'possui_telefone_trabalho',
                          'meses_no_trabalho', 
                          'profissao', 
                          'ocupacao', 
                          'profissao_companheiro',
                          'grau_instrucao_companheiro', 
                          'local_onde_reside',
                          'local_onde_trabalha', 
                          'forma_envio_solicitacao_correio', 
                          'forma_envio_solicitacao_internet',
                          'forma_envio_solicitacao_presencial',
                          'inadimplente'
                         ]

dados = dados[atributos_selecionados]

In [270]:
# Separar nos conjuntos de treino e teste
treino = dados.iloc[:20000, :]
teste = dados.iloc[20000:, :]

In [271]:
treino_emb = treino.sample(frac=1, random_state=12345)
teste_emb = teste.sample(frac=1, random_state=12345)

In [272]:
X_treino = treino_emb.loc[:, treino_emb.columns!='inadimplente']
X_teste = teste_emb.loc[:, treino_emb.columns!='inadimplente']

y_treino = treino_emb['inadimplente']
y_teste = teste_emb['inadimplente']

In [273]:
# Ajustar a escala dos atributos nos conjuntos de treino e teste
from sklearn.preprocessing import MinMaxScaler

ajustador_de_escala = MinMaxScaler() 

ajustador_de_escala.fit(X_treino)

X_treino = ajustador_de_escala.transform(X_treino)
X_teste = ajustador_de_escala.transform(X_teste)

In [274]:
# Treinar o classificador
from sklearn.neighbors import KNeighborsClassifier
classificador = KNeighborsClassifier(n_neighbors=10)
classificador = classificador.fit(X_treino, y_treino)

In [275]:
y_resposta_teste = classificador.predict(X_teste)

# Verificar a acurácia
total_amostras = len(y_teste)
acertos = sum(y_resposta_teste==y_teste)
erros = sum(y_resposta_teste!=y_teste)

(total_amostras, acertos, erros)
print('Acurácia = %.1f %%' % (acertos * 100/total_amostras))

Acurácia = 60.4 %


In [276]:
print(" K   TESTE")
print("--   -----")
for k in range(3, 20, 2):
    classificador = KNeighborsClassifier(   
        n_neighbors=k,
        weights='distance',
        p=1
    )
    classificador = classificador.fit(X_treino, y_treino)
    
    y_resposta_teste = classificador.predict(X_teste)
    
    acuracia_teste = sum(y_resposta_teste==y_teste) / len(y_teste)
    
    print(
        '%2d' %k,
        '%6.1f' % (100 * acuracia_teste)
        )

 K   TESTE
--   -----
 3   51.0
 5   51.0
 7   51.0
 9   51.2
11   51.5
13   51.5
15   51.6
17   51.2
19   51.2


In [277]:
# Acurácia
acuracia_teste = sum(y_resposta_teste==y_teste) / len(y_teste)
100 * acuracia_teste

51.2

In [278]:
# Pegar os id_solicitantes em ordem 
y_id = pd.DataFrame(y_teste.index, columns=['id_solicitante'])

In [279]:
# Junto os id's com os resultados e transformo em um único DataFrame
y_resposta = pd.DataFrame(y_resposta_teste, columns=['inadimplente'])
y_resposta = pd.concat([y_id, y_resposta.astype('int64')], axis=1)
y_resposta

,id_solicitante,inadimplente
0,23184,1
1,21072,1
2,22641,1
3,22283,1
4,21596,0
...,...,...
4995,23498,1
4996,23493,1
4997,22178,0
4998,23558,0


In [280]:
# Obter o csv com inadimplentes e id_solicitante
y_resposta.to_csv("resp_trab1.csv", index=False)